In [1]:
from unsloth import FastLanguageModel
import torch, psutil, os, gc
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import requests
from MyModule import get_current_time, get_current_date, get_weather, search_address, get_alarms, set_alarms, delete_alarms

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/user/miniconda3/envs/unsloth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model_name = "../model/kanana-1.5-2.1b-instruct-2505"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj",
        "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.1.3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
dataset1 = load_dataset("json", data_files="./data/train_data_singleturn.jsonl")
dataset2 = load_dataset("json", data_files="./data/train_data_function_call_kanana.jsonl")
dataset3 = load_dataset("json", data_files="./data/train_data_function_call_alarm.jsonl")

dataset = concatenate_datasets([
    dataset1['train'],
    dataset2['train'],
    dataset3['train'],
    dataset3['train'],
]).shuffle(seed=42)
dataset[0]

Generating train split: 370 examples [00:00, 4355.14 examples/s]


{'messages': [{'role': 'user', 'content': '이번주 일요일 아침 9시 교회 가기'},
  {'role': 'assistant',
   'content': '<function=set_alarms>{"message": "교회 가기", "year": null, "month": null, "day": null, "hour": 9, "minute": 0, "relative_day": 0, "week_offset": 0, "day_of_week": "일요일", "is_ampm_specified": true}</function>'},
  {'role': 'tool',
   'content': '{"status": "success", "time": "20260125T0900", "day_of_week": "일요일", "message_content": "교회 가기", "message": "알람을 성공적으로 설정했습니다."} '},
  {'role': 'assistant', 'content': "1월 25일 일요일 오전 9시에 '교회 가기' 알람을 설정했습니다."}]}

In [6]:
TOOLS = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_weather": get_weather,
    "search_address": search_address,
    "get_alarms": get_alarms,
    "set_alarms": set_alarms,
    "delete_alarm": delete_alarms,
}

TOOL_DEFINITIONS = [
    {
        "name": "get_current_time",
        "description": "현재 시간을 반환합니다.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_date",
        "description": "현재 날짜를 반환합니다.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_weather",
        "description": "특정 지역의 날씨 정보를 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "지역 이름"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_address",
        "description": "주소를 검색합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "검색어"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_alarms",
        "description": "현재 등록된 모든 알람 목록을 확인합니다. (매개변수 없음)"
    },
    {
        "name": "set_alarms",
        "description": "새로운 알람을 예약합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {
                    "type": "string",
                    "description": "알람 시 알려줄 내용"
                },
                "year": {
                    "type": "integer",
                    "description": "연도 (4자리 숫자)"
                },
                "month": {
                    "type": "integer",
                    "description": "월 (1-12)"
                },
                "day": {
                    "type": "integer",
                    "description": "일 (1-31)"
                },
                "hour": {
                    "type": "integer",
                    "description": "시 (0-23)"
                },
                "minute": {
                    "type": "integer",
                    "description": "분 (0-59)"
                },
                "relative_day": {
                    "type": "integer",
                    "description": "오늘 기준 날짜 차이. 오늘=0, 내일=1, 모레=2, 글피=3 등등"
                },
                "week_offset": {
                    "type": "integer",
                    "description": "주 단위 차이. 이번 주=0, 다음 주=1"
                },
                "day_of_week": {
                    "type": "string",
                    "description": "요일 (예: '월요일', '화요일', ..., '일요일')"
                },
                "is_ampm_specified": {
                    "type": "boolean",
                    "description": "사용자가 '오전', '오후', '아침', '밤' 등 시간을 지정했다면 True, 지정하지 않았다면 False를 주세요."
                }
            },
            "required": ["hour"]
        }
    },
    {
        "name": "delete_alarms",
        "description": "등록된 알람을 삭제합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {
                    "type": "string",
                    "description": "삭제할 알람의 메시지 내용"
                },
                "year": {
                    "type": "integer",
                    "description": "연도 (4자리 숫자)"
                },
                "month": {
                    "type": "integer",
                    "description": "월 (1-12)"
                },
                "day": {
                    "type": "integer",
                    "description": "일 (1-31)"
                },
                "hour": {
                    "type": "integer",
                    "description": "시 (0-23)"
                },
                "minute": {
                    "type": "integer",
                    "description": "분 (0-59)"
                },
                "relative_day": {
                    "type": "integer",
                    "description": "오늘 기준 날짜 차이. 오늘=0, 내일=1, 모레=2, 글피=3 등등"
                },
                "week_offset": {
                    "type": "integer",
                    "description": "주 단위 차이. 이번 주=0, 다음 주=1"
                },
                "day_of_week": {
                    "type": "string",
                    "description": "요일 (예: '월요일', '화요일', ..., '일요일')"
                },
                "is_ampm_specified": {
                    "type": "boolean",
                    "description": "사용자가 '오전', '오후', '아침', '밤' 등 시간을 지정했다면 True, 지정하지 않았다면 False를 주세요."
                }
            }
        }
    }
]

In [7]:
# assistant_only_loss = False일 경우
def formatting_prompts_func(examples):
    
    texts = []

    for message in examples['messages']:
        text = tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=False,
            tools=TOOL_DEFINITIONS
        )

        texts.append(text)

    return {'text': texts}

formatted_dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)
print(formatted_dataset[0])

split_dataset = formatted_dataset.train_test_split(test_size=0.1)
train_ds = split_dataset["train"]
eval_ds = split_dataset["test"]

Map: 100%|██████████| 5982/5982 [00:00<00:00, 6677.71 examples/s]


{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\n\nYou have access to the following functions:\n\nUse the function \'get_current_time\' to \'현재 시간을 반환합니다.\'\n{"name": "get_current_time", "description": "현재 시간을 반환합니다.", "parameters": {"type": "object", "properties": {}, "required": []}}\n\nUse the function \'get_current_date\' to \'현재 날짜를 반환합니다.\'\n{"name": "get_current_date", "description": "현재 날짜를 반환합니다.", "parameters": {"type": "object", "properties": {}, "required": []}}\n\nUse the function \'get_weather\' to \'특정 지역의 날씨 정보를 조회합니다.\'\n{"name": "get_weather", "description": "특정 지역의 날씨 정보를 조회합니다.", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "지역 이름"}}, "required": ["city"]}}\n\nUse the function \'search_address\' to \'주소를 검색합니다.\'\n{"name": "search_address", "description": "주소를 검색합니다.", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "검색어"}}, "required": ["location"]}}

In [8]:
sample_text = formatted_dataset[0]["text"]

tokens = tokenizer(
    sample_text,
    add_special_tokens=False,
    return_attention_mask=False,
    return_tensors=None
)

print("토큰 개수:", len(tokens["input_ids"]))

토큰 개수: 1353


In [14]:
# # assistant_only_loss = True일 경우
# def formatting_prompts_func(examples):
#     if len(examples["messages"]) > 0 and isinstance(examples["messages"][0], list):
#         messages_list = examples["messages"] # 예) {"messages": [ [채팅1], [채팅2], [채팅3] ]}
#     else:
#         # 단일 샘플 처리 (SFTTrainer 검증용) 
#         messages_list = [examples["messages"]] # 예) {"messages": [채팅1]}

#     texts = []
#     for message in messages_list:
#         text = tokenizer.apply_chat_template(
#             message,
#             tokenize=False,
#             add_generation_prompt=False,
#             tools=TOOL_DEFINITIONS
#         )
#         texts.append(text)

#     # SFTTrainer의 formatting_func는 반드시 '문장들의 리스트'를 반환해야 한다.
#     return texts

# split_dataset2 = dataset.train_test_split(test_size=0.1)
# train_ds2 = split_dataset2["train"]
# eval_ds2 = split_dataset2["test"]

In [9]:
args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 400,
        num_train_epochs=1,
        learning_rate = 5e-5, 
        bf16 = True,
        seed = 1234,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        output_dir = "outputs",

        logging_steps=200,
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,

        # SFTConfig의 파라미터
        packing=False,
        dataset_text_field="text",
        # model_init_kwargs={"trust_remote_code": True},
        # assistant_only_loss=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
    # formatting_func=formatting_prompts_func,
    args=args
)

Unsloth: Tokenizing ["text"] (num_proc=4): 100%|██████████| 599/599 [00:00<00:00, -601.99 examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [10]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,383 | Num Epochs = 1 | Total steps = 673
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 23,003,136 of 2,109,987,840 (1.09% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
200,0.257500,0.101896
400,0.097000,0.093805
600,0.090400,0.089616


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


KeyboardInterrupt: 

### 테스트

In [1]:
import torch, psutil, os, gc, re, json
from datetime import datetime, timedelta
from unsloth import FastLanguageModel
from transformers import GenerationConfig, TextIteratorStreamer
from peft import PeftModel
from threading import Thread
from memory_monitor import MemoryMonitor
import requests
from MyModule import get_current_time, get_current_date, get_weather, search_address, get_alarms, set_alarms, delete_alarms

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/user/miniconda3/envs/unsloth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
monitor = MemoryMonitor(device_index=0)
torch.cuda.empty_cache()
gc.collect()

monitor.print_usage(label="Before Loading")

# 모델 로드
model_name = "../model/kanana-1.5-2.1b-instruct-2505"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

model = PeftModel.from_pretrained(model, "./outputs/checkpoint-600")

# 추론 모드로 전환
FastLanguageModel.for_inference(model)

monitor.print_usage(label="After Loading")

# 1 token 생성
prompt = "안녕하세요."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
_ = model.generate(**inputs, max_new_tokens=1)
monitor.print_usage(label="After Generating")

[Before Loading] RAM: 977.14 MB | VRAM: 0.00 MB
==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
[After Loading] RAM: 1333.55 MB | VRAM: 0.00 MB
[After Generating] RAM: 1695.93 MB | VRAM: 0.00 MB


In [3]:
TOOLS = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_weather": get_weather,
    "search_address": search_address,
    "get_alarms": get_alarms,
    "set_alarms": set_alarms,
    "delete_alarm": delete_alarms,
}

TOOL_DEFINITIONS = [
    {
        "name": "get_current_time",
        "description": "현재 시간을 반환합니다.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_date",
        "description": "현재 날짜를 반환합니다.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_weather",
        "description": "특정 지역의 날씨 정보를 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "지역 이름"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_address",
        "description": "주소를 검색합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "검색어"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_alarms",
        "description": "현재 등록된 모든 알람 목록을 확인합니다. (매개변수 없음)"
    },
    {
        "name": "set_alarms",
        "description": "새로운 알람을 예약합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {
                    "type": "string",
                    "description": "알람 시 알려줄 내용"
                },
                "year": {
                    "type": "integer",
                    "description": "연도 (4자리 숫자)"
                },
                "month": {
                    "type": "integer",
                    "description": "월 (1-12)"
                },
                "day": {
                    "type": "integer",
                    "description": "일 (1-31)"
                },
                "hour": {
                    "type": "integer",
                    "description": "시 (0-23)"
                },
                "minute": {
                    "type": "integer",
                    "description": "분 (0-59)"
                },
                "relative_day": {
                    "type": "integer",
                    "description": "오늘 기준 날짜 차이. 오늘=0, 내일=1, 모레=2, 글피=3 등등"
                },
                "week_offset": {
                    "type": "integer",
                    "description": "주 단위 차이. 이번 주=0, 다음 주=1"
                },
                "day_of_week": {
                    "type": "string",
                    "description": "요일 (예: '월요일', '화요일', ..., '일요일')"
                },
                "is_ampm_specified": {
                    "type": "boolean",
                    "description": "사용자가 '오전', '오후', '아침', '밤' 등 시간을 명확히 지정했는지 여부"
                }
            },
            "required": ["hour"]
        }
    },
    {
        "name": "delete_alarms",
        "description": "등록된 알람을 삭제합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {
                    "type": "string",
                    "description": "삭제할 알람의 메시지 내용"
                },
                "year": {
                    "type": "integer",
                    "description": "연도 (4자리 숫자)"
                },
                "month": {
                    "type": "integer",
                    "description": "월 (1-12)"
                },
                "day": {
                    "type": "integer",
                    "description": "일 (1-31)"
                },
                "hour": {
                    "type": "integer",
                    "description": "시 (0-23)"
                },
                "minute": {
                    "type": "integer",
                    "description": "분 (0-59)"
                },
                "relative_day": {
                    "type": "integer",
                    "description": "오늘 기준 날짜 차이. 오늘=0, 내일=1, 모레=2, 글피=3 등등"
                },
                "week_offset": {
                    "type": "integer",
                    "description": "주 단위 차이. 이번 주=0, 다음 주=1"
                },
                "day_of_week": {
                    "type": "string",
                    "description": "요일 (예: '월요일', '화요일', ..., '일요일')"
                },
                "is_ampm_specified": {
                    "type": "boolean",
                    "description": "사용자가 '오전', '오후', '아침', '밤' 등 시간을 명확히 지정했는지 여부"
                }
            }
        }
    }
]

In [4]:
generation_config = GenerationConfig(
    max_new_tokens=2048,  # 생성할 최대 토큰 수
    temperature=0.1,
    top_p=0.95,
    top_k=50,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,  # 끝 토큰 명시
    pad_token_id=tokenizer.pad_token_id,
)

def stream_with_langchain(question):
    messages = [
        {"role": "user", "content": question}
    ]
    
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        tools=TOOL_DEFINITIONS
    )

    inputs = tokenizer(
        formatted_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048,
    )

    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    
    streamer = TextIteratorStreamer(
        tokenizer,
        skip_special_tokens=True,   # <|end|> 같은 특수 토큰 건너뛰기
        skip_prompt=True            # 입력 프롬프트 건너뛰기 (응답만 보기)
    )           
    
    # Thread가 있어야 메인 스레드에서 백그라운드로 generate()가 실행되고
    # 서브 스레드에서 streamer로 토큰을 실시간으로 읽을 수 있음
    # 따라서 동시에 병렬로 처리 가능

    thread = Thread(
        target=model.generate, 
        kwargs=dict(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            streamer=streamer,
        )
    )
    thread.start()
    
    full_response = ""
    for text in streamer:
        print(text, end="", flush=True)
        full_response += text
    
    thread.join()
    print()     # 줄바꿈

    pattern = r"<function=(\w+)>(.+?)</function>"
    match = re.search(pattern, full_response)

    if match:
        # 함수 호출 형태 감지됨
        func_name = match.group(1)  # get_weather
        args_str = match.group(2) # "location": "Seoul"

        args = json.loads(args_str)

        tool_result = TOOLS[func_name](**args)

        messages.append({
            "role": "assistant",
            "content": full_response
        })
        messages.append({
            "role": "tool",
            "content": json.dumps(tool_result, ensure_ascii=False)
        })

        formatted_text_2 = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        inputs2 = tokenizer(
            formatted_text_2,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=2048,
        )

        streamer2 = TextIteratorStreamer(
            tokenizer,
            skip_special_tokens=True,   # <|end|> 같은 특수 토큰 건너뛰기
            skip_prompt=True            # 입력 프롬프트 건너뛰기 (응답만 보기)
        )

        thread2 = Thread(
            target=model.generate, 
            kwargs=dict(
                input_ids=inputs2["input_ids"].to(model.device),
                attention_mask=inputs2["attention_mask"].to(model.device),
                generation_config=generation_config,
                streamer=streamer2,
            )
        )
        thread2.start()
        
        final_response = ""
        for text in streamer2:
            print(text, end="", flush=True)
            final_response += text
        
        thread2.join()
        print()

        return final_response

    return full_response

In [10]:
question = "알람 좀 확인해줘"
print(f"Q: {question}")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)

Q: 알람 좀 확인해줘
A: <function=get_alarms>{}</function>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [41]:
question = "달서구 날씨 어때? 비오려나? 우산을 안 챙겼는데 걱정이네"
print(f"Q: {question}")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)

Q: 달서구 날씨 어때? 비오려나? 우산을 안 챙겼는데 걱정이네
A: <function=get_weather>{"city":"달서구"}</function>

</function>
{'city': '대구 달서구', 'temperature': '4도', 'sky_condition': '흐림', 'precipitation_type': '없음'}
현재 대구 달서구는 4도이며 흐린 날씨입니다. 비 소식은 없지만, 갑자기 흐려질 수 있으니 우산을 챙기는 게 좋겠어요.


### GGUF 변환

In [1]:
from unsloth import FastLanguageModel

# 모델 및 아답터 로드 (한 번에 불러오기)
adapter_path = "./outputs/kanana-tools-checkpoint-600" # 학습된 아답터

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = adapter_path, # 아답터 경로를 넣으면 베이스 모델까지 자동으로 찾아서 합친다.
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)

# 추론 모드로 변환 (필수)
FastLanguageModel.for_inference(model)

# 젯슨용 GGUF로 저장하기
model.save_pretrained_merged(
    "../kanana_merged_temp",    # 저장될 폴더명/파일명
    tokenizer,
    save_method = "merged_16bit"
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/user/miniconda3/envs/unsloth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Detected local model directory: /home/user/work/model/kanana-1.5-2.1b-instruct-2505
Found HuggingFace hub cache directory: /home/user/.cache/huggingface/hub


Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [01:31<00:00, 91.66s/it]


Copied model.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [02:38<00:00, 158.13s/it]


Unsloth: Merge process complete. Saved to `/home/user/work/kanana_merged_temp`


In [9]:
# 1. llama.cpp 폴더로 이동 (주피터 세션 내 경로 변경은 %cd 사용)
%cd /home/user/work/llama.cpp

# 2. f16 gguf로 변환 (!를 사용하여 셸 명령어 실행)
!python convert_hf_to_gguf.py ../kanana_merged_temp --outfile ../kanana_f16.gguf

# 3. q4_k_m gguf로 변환
!./build/bin/llama-quantize ../kanana_f16.gguf ../kanana_q4_k_m.gguf q4_k_m

/home/user/work/llama.cpp
INFO:hf-to-gguf:Loading model: kanana_merged_temp
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:hf-to-gguf:heuristics detected bfloat16 tensor dtype, setting --outtype bf16
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:output.weight,               torch.bfloat16 --> BF16, shape = {1792, 128259}
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> BF16, shape = {1792, 128259}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {1792}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> BF16, shape = {8064, 1792}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> BF16, shape = {1792, 8064}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> BF16, shape = {1792, 8064}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {17